In [1]:
!pip install transformers -U


In [2]:
import pandas as pd


In [3]:
data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")
data.head()

<ipython-input-3-dfd017dc2c35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")


,Abstract,Question,Answer,Label,TemporalAspect
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes,Frequency
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes,Frequency
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes,Typical Time
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes,Typical Time
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes,Event Duration


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification


In [5]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [6]:
model = model.to('cuda')


In [7]:
import numpy as np  # Import numpy for NaN handling

# Map string labels to numerical values
label_mapping = {"yes": 1, "no": 0}
data["Label"] = data["Label"].map(label_mapping)

data["Abstract"] = data["Abstract"].fillna("")
data["Question"] = data["Question"].fillna("")
data["Answer"] = data["Answer"].fillna("")

# Concatenate the strings for context, question, and answer
X = ["[CLS] " + str(abstract) + " [SEP] " + str(question) + " [SEP] " + str(answer) + " [SEP]" for abstract, question, answer in zip(data["Abstract"], data["Question"], data["Answer"])]

y = list(data["Label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenizing a single sequence (context, question, answer concatenated)
X_train_tokenized = tokenizer(
    X_train,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)

X_val_tokenized = tokenizer(
    X_val,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)



In [8]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [9]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [10]:

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.5 MB/s eta 0:00:00


In [13]:
import optuna
from transformers import Trainer, TrainingArguments
import torch



def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 3, 10)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    args = TrainingArguments(
        output_dir="output",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate
        # Add other hyperparameters here
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    try:
        trainer.train()
        result = trainer.evaluate()
        return result['eval_loss']
    except RuntimeError as e:
        print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
        torch.cuda.empty_cache()
        return float('inf')

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

best_params = study.best_params


[I 2023-12-19 18:13:30,938] A new study created in memory with name: no-name-635687c1-7ceb-4dca-9b8e-137d66fb55c6
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.722300
1000,0.700200
1500,0.697700
2000,0.688800
2500,0.685500
3000,0.687200
3500,0.669100
4000,0.668800
4500,0.638700
5000,0.641000


<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tens

[I 2023-12-19 18:27:21,948] Trial 0 finished with value: 0.8591964244842529 and parameters: {'num_epochs': 6, 'batch_size': 4, 'learning_rate': 2.4759623489260402e-05}. Best is trial 0 with value: 0.8591964244842529.


<class 'transformers.trainer_utils.EvalPrediction'>


...

<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.684700
1000,0.685000


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-12-19 19:14:37,739] Trial 4 finished with value: 0.684695839881897 and parameters: {'num_epochs': 5, 'batch_size': 16, 'learning_rate': 0.00010311009883125963}. Best is trial 4 with value: 0.684695839881897.


<class 'transformers.trainer_utils.EvalPrediction'>


In [14]:
import json

# Save best_params to a JSON file
with open('best_hyperparameters(RoBERTa).json', 'w') as file:
    json.dump(best_params, file)

In [ ]:
# Define Trainer
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.713300
1000,0.696400
1500,0.694500
2000,0.688000
2500,0.683700
3000,0.676400
3500,0.665800
4000,0.649100
4500,0.627000
5000,0.594800


<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tens

TrainOutput(global_step=5000, training_loss=0.6688922119140625, metrics={'train_runtime': 943.3754, 'train_samples_per_second': 42.401, 'train_steps_per_second': 5.3, 'total_flos': 1993924271040000.0, 'train_loss': 0.6688922119140625, 'epoch': 10.0})

In [ ]:
trainer.evaluate()


<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.7772216796875,
 'eval_accuracy': 0.541,
 'eval_precision': 0.4666666666666667,
 'eval_recall': 0.38620689655172413,
 'eval_f1': 0.4226415094339622,
 'eval_runtime': 5.2228,
 'eval_samples_per_second': 191.466,
 'eval_steps_per_second': 23.933,
 'epoch': 10.0}

In [ ]:
np.set_printoptions(suppress=True)


In [ ]:
context = "Wallace, 38, called Gastonia home from the age of 8 until she graduated from Hunter Huss High School in 1983.	"
question = "How often does Brewer talk about the legal system marketplace?	"
answer = "pushkin liked to speak at high noon usually	"
input_sequence = f"[CLS] {context} [SEP] {question} [SEP] {answer} [SEP]"

# text = "go to hell"
tokenized_input = tokenizer(input_sequence,padding = True, truncation = True, return_tensors='pt').to('cuda')
# Make predictions
with torch.no_grad():
    outputs = model(**tokenized_input)

# Get the predicted label
logits = outputs.logits
predicted_label = "yes" if logits.argmax() == 1 else "no"
print("Predicted Label:", predicted_label)

Predicted Label: no


In [ ]:
! pip install accelerate -U
! pip install -U transformers
import os
os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
